# 1. Ambil Data Dari Gdelt, Ambil yang Nopember Saja

In [2]:
import requests
import lxml.html as lh

gdelt_base_url = 'http://data.gdeltproject.org/events/'

# get the list of all the links on the gdelt file page
page = requests.get(gdelt_base_url+'index.html')
doc = lh.fromstring(page.content)
link_list = doc.xpath("//*/ul/li/a/@href")

# separate out those links that begin with four digits 
file_list = [x for x in link_list if (str(x[0:6]) == '201812')]
print(file_list)
#print("Done!")

['20181202.export.CSV.zip', '20181201.export.CSV.zip']


# 2. Masukkan Data ke Dalam Format TSV untuk Data Dari Negara Indonesia

In [3]:
infilecounter = 0
outfilecounter = 0

import os.path
import urllib
import zipfile
import glob
import operator

local_path = 'D:/GDELT/'
dir_country = 'country'
fips_country_code = 'ID'

for compressed_file in file_list[infilecounter:]:
    print(compressed_file),
    
    # if we dont have the compressed file stored locally, go get it. Keep trying if necessary.
    while not os.path.isfile(local_path+compressed_file): 
        print('Downloading....')
        urllib.request.urlretrieve(url=gdelt_base_url + compressed_file,
                           filename=local_path + compressed_file)
    
    # extract the contents of the compressed file to a temporary directory    
    print('extracting,')
    z = zipfile.ZipFile(file=local_path+compressed_file, mode='r')    
    z.extractall(path=local_path+'tmp/')
    
    # parse each of the csv files in the working directory, 
    print('parsing,')
    
    for infile_name in glob.glob(local_path+'tmp/*'):
        outfile_name = local_path+dir_country+'/'+fips_country_code+'%04i.tsv'%outfilecounter
        
        try:
            # Create target Directory
            os.mkdir(local_path+dir_country)
            print("Directory " , dir_country ,  " Created ") 
        except FileExistsError:
            print("Directory " , dir_country ,  " already exists")
        
        # open the infile and outfile
        print(infile_name)
        with open(infile_name,mode='r') as infile, open(outfile_name,mode='w') as outfile:
            try:
                for line in infile:
                    # extract lines with our interest country code
                    if fips_country_code in operator.itemgetter(51, 37, 44)(line.split('\t')):    
                        outfile.write(line)
            except:
                print("")
            outfilecounter +=1
            
        # delete the temporary file
        os.remove(infile_name)
    infilecounter +=1
    print('done')

20181202.export.CSV.zip
Downloading....
extracting,
parsing,
Directory  country  already exists
D:/GDELT/tmp\20181202.export.CSV

done
20181201.export.CSV.zip
Downloading....
extracting,
parsing,
Directory  country  already exists
D:/GDELT/tmp\20181201.export.CSV

done


# 3. Masukkan Data Dari TSV ke Database

In [4]:
import glob
import pymysql
import pandas as pd

host = '127.0.0.1'
port = "3306"
user = 'root'
password = ''
database = 'news'

conn = pymysql.connect(
    host=host,
    port=int(port),
    user=user,
    passwd=password,
    db=database,
    charset='utf8mb4')

def run(sql):
    df = pd.read_sql_query(sql,conn)
    return df
x = conn.cursor()

# Build DataFrames from each of the intermediary files
files = glob.glob(local_path+'country/'+fips_country_code+'*')
DFlist = []
print("We are working on it, please take your coffee...")  
for active_file in files:
    with open(active_file,mode='r') as xfile:
        for line in xfile:
            kolom = line.split("\t")
            kolomID = kolom[0]
            kolomURL = kolom[57]
       
            sql = "INSERT INTO gdelt_content201812 (GLOBALEVENTID,URL) SELECT * FROM (SELECT '"+kolomID+"','"+kolomURL+"') AS tmp WHERE NOT EXISTS (SELECT GLOBALEVENTID FROM gdelt_content WHERE GLOBALEVENTID = '"+kolomID+"') LIMIT 1"
            #print(sql)
            x.execute(sql)
            conn.commit()
print("Done! All have finished")
    
    
    



We are working on it, please take your coffee...
Done! All have finished


# 4. Update Title dan Content data di Database 

In [5]:
from newspaper import Article
import pymysql.cursors
import sys

offset = sys.argv[1]
limit = sys.argv[2]
# year = sys.argv[3]

connection = pymysql.connect(host='localhost',
                             user='root',
                             password='',
                             db='news',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

hdr = {'User-Agent': 'Mozilla/5.0'}
print('Take your coffee again! We will do this job for you...')
with connection.cursor() as cursor:
    sql = "SELECT * FROM `gdelt_content201812` WHERE (TITLE IS NULL)"
    cursor.execute(sql)
    print(sql)
    result = cursor.fetchall()
    for r in result:
        try:
            r["URL"] = r["URL"].split('<UDIV>')
            print(r["URL"][0])
            article = Article(r["URL"][0])
            article.download()
            article.parse()
            with connection.cursor() as cursor:
                sql = "UPDATE `gdelt_content201812` SET title=%s,content=%s WHERE GLOBALEVENTID=%s"
                cursor.execute(sql, (article.title.encode('utf-8'), article.text.encode('utf-8'),r["GLOBALEVENTID"]))
                connection.commit()
        except:
            pass

print("Done!")

Take your coffee again! We will do this job for you...
SELECT * FROM `gdelt_content201812` WHERE (TITLE IS NULL)
http://infosurhoy.com/cocoon/saii/xhtml/en_GB/science/science-over-one-third-of-indonesias-coral-reefs-in-dangerous-state-research-finds-report/

http://infosurhoy.com/cocoon/saii/xhtml/en_GB/science/science-over-one-third-of-indonesias-coral-reefs-in-dangerous-state-research-finds-report/

https://www.chron.com/neighborhood/news/houston-texas/houston/article/Indonesians-take-center-stage-at-Jones-Plaza-13437038.php

https://www.chron.com/neighborhood/news/houston-texas/houston/article/Indonesians-take-center-stage-at-Jones-Plaza-13437038.php

https://www.chron.com/neighborhood/news/houston-texas/houston/article/Indonesians-take-center-stage-at-Jones-Plaza-13437038.php

https://www.chron.com/neighborhood/news/houston-texas/houston/article/Indonesians-take-center-stage-at-Jones-Plaza-13437038.php

https://www.chron.com/neighborhood/news/houston-texas/houston/article/Indonesia

https://www.fijitimes.com/png-and-indonesia-seek-to-develop-border-trade/

Done!
